In [1]:
!pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of motor to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━

Reproduced with code from this excellent [blog](https://towardsdatascience.com/faster-r-cnn-object-detection-implemented-by-keras-for-custom-data-from-googles-open-images-125f62b9141a)

In [9]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tf.__version__

'2.17.0'

In [62]:
import os
import csv
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import random
from google.colab import drive
drive.mount('/content/drive')

# Define the base path where you want to save CSVs and other files
base_path = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2'

# Set global seed for reproducibility
random.seed(42)

# Common label map between VOC and COCO, focusing only on the common categories
common_label_map = {
    'person': 1,
    'bicycle': 2,
    'car': 3,
    'motorcycle': 4,  # 'motorbike' in VOC
    'airplane': 5,    # 'aeroplane' in VOC
    'bus': 6,
    'train': 7,
    'truck': 8,
    'boat': 9,
    'bird': 10,
    'cat': 11,
    'dog': 12,
    'horse': 13,
    'sheep': 14,
    'cow': 15,
    'elephant': 16,
    'bear': 17,
    'zebra': 18,
    'giraffe': 19,
    'chair': 20,
    'couch': 21,  # 'sofa' in VOC
    'potted plant': 22,  # 'pottedplant' in VOC
    'dining table': 23,  # 'diningtable' in VOC
    'tv': 24    # 'tvmonitor' in VOC
}

# Load the COCO-2017 dataset using FiftyOne
print("Loading the COCO-2017 dataset...")
coco_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train", "validation"],
    max_samples=5000,
    dataset_name="coco-2017"
)


# Mapping from COCO class names to VOC class names
coco_to_voc_mapping = {
    'airplane': 'aeroplane',
    'motorcycle': 'motorbike',
    'sofa': 'couch',
    'potted plant': 'pottedplant',
    'dining table': 'diningtable',
    'tv': 'tvmonitor',
}

# Print the labels in the dataset before any processing
print("\nLabels before any processing:")
original_labels = coco_dataset.distinct("ground_truth.detections.label")
print(f"Number of unique labels before processing: {len(original_labels)}")
for label in original_labels:
    print(f"Original label: {label}")

# Combine original COCO labels and VOC-mapped labels for filtering
combined_labels = set(common_label_map.keys()).union(set(coco_to_voc_mapping.keys())).union(set(coco_to_voc_mapping.values()))
print("\nCombined labels for filtering:")
print(combined_labels)

# Filter for common categories (both COCO and VOC names)
print("\nFiltering dataset for common categories...")
label_field = F("label")
coco_dataset = coco_dataset.filter_labels("ground_truth", label_field.is_in(combined_labels))
print(f"Number of samples after filtering: {len(coco_dataset)}")

# Print the filtered labels
filtered_labels = coco_dataset.distinct("ground_truth.detections.label")
print(f"Number of unique labels after filtering: {len(filtered_labels)}")
print("Labels after filtering:")
for label in filtered_labels:
    print(f"Filtered label: {label}")

# Replace COCO class names with VOC equivalents
print("\nReplacing COCO class names with VOC equivalents...")
for sample in coco_dataset:
    detections = sample.ground_truth.detections
    for detection in detections:
        original_label = detection.label
        if original_label in coco_to_voc_mapping:
            new_label = coco_to_voc_mapping[original_label]
            detection.label = new_label
            print(f"Updated label: {original_label} -> {new_label}")
    sample.save()

# Print the labels in the dataset after renaming
print("\nLabels after renaming to VOC equivalents:")
updated_labels = coco_dataset.distinct("ground_truth.detections.label")
print(f"Number of unique labels after renaming: {len(updated_labels)}")
for label in updated_labels:
    print(f"Updated label: {label}")

# Step to balance class distributions
# Count the number of samples per class
class_counts = coco_dataset.count_values("ground_truth.detections.label")
print(f"Class counts before margin: {class_counts}")

# Calculate the minimum class count in the dataset
min_class_count = min(class_counts.values())

# Define a margin (e.g., 50% above the minimum)
margin = 0.5
target_instances = int(min_class_count * (1 + margin))

print(f"Minimum class count: {min_class_count}")
print(f"Target instances per class: {target_instances}")

# Adjust the class distribution in the COCO dataset
for cls in filtered_labels:
    # Filter the dataset for samples that contain the specific class (label)
    class_view = coco_dataset.filter_labels("ground_truth", F("label") == cls)

    # Debug: Print the number of instances before filtering
    print(f"Class: {cls}, Instance Count: {len(class_view)}")

    # Check if the class has more instances than the target number
    if len(class_view) > target_instances:
        # Select a random subset of the class to keep
        to_keep = class_view.take(target_instances, seed=42)

        # Collect the IDs of the samples to keep
        to_keep_ids = [sample.id for sample in to_keep]

        # Exclude the samples not in to_keep from the coco_dataset
        coco_dataset = coco_dataset.exclude([sample.id for sample in class_view if sample.id not in to_keep_ids])

        # Debug: Print the reduction details
        print(f"Reduced {cls} from {len(class_view)} to {len(to_keep)} samples")
    else:
        # Debug: Print when no reduction is applied
        print(f"No reduction needed for {cls}, instances within target ({len(class_view)} <= {target_instances})")

# After adjustment, re-check the class counts
new_class_counts = coco_dataset.count_values("ground_truth.detections.label")
print(f"New class counts after adjustment: {new_class_counts}")

# Ensure directories exist
os.makedirs(base_path, exist_ok=True)

# Prepare paths for CSV files
images_csv = os.path.join(base_path, "train-images-boxable.csv")
annotations_csv = os.path.join(base_path, "train-annotations-bbox.csv")
classes_csv = os.path.join(base_path, "class-descriptions-boxable.csv")

# 1. Create `train-images-boxable.csv` with image names and file paths
print("\nWriting image paths to CSV...")
with open(images_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ImageName", "ImageURL"])

    for sample in coco_dataset:
        image_name = os.path.basename(sample.filepath)
        image_url = sample.filepath
        writer.writerow([image_name, image_url])
print(f"Finished writing {images_csv}")

# 2. Create `class-descriptions-boxable.csv` with updated class names and LabelName as IDs
print("\nWriting updated class descriptions to CSV...")
with open(classes_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["LabelName", "ClassName"])

    # Combine both original COCO and new VOC labels for writing
    for original_name, id_value in common_label_map.items():
        if original_name in coco_to_voc_mapping:
            class_name = coco_to_voc_mapping[original_name]  # Use the VOC name
        else:
            class_name = original_name  # Use the COCO name if no VOC mapping

        writer.writerow([id_value, class_name])
        print(f"Processed {id_value} -> {class_name}")
print("Finished writing class-descriptions-boxable.csv")

# 3. Create `train-annotations-bbox.csv` with bounding box annotations using updated labels
print("\nWriting bounding box annotations to CSV...")
with open(annotations_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ImageID", "LabelName", "XMin", "YMin", "XMax", "YMax"])

    for sample in coco_dataset:
        image_name = os.path.basename(sample.filepath)
        detections = sample.ground_truth.detections

        for detection in detections:
            label = detection.label
            # We assume that by this point, the label has already been updated and is in its final form
            if label in updated_labels:
                # Use the updated label directly without remapping
                label_id = common_label_map.get(label) or next(
                    (id_value for original_name, id_value in common_label_map.items()
                     if coco_to_voc_mapping.get(original_name) == label), None)

                if label_id is not None:
                    bbox = detection.bounding_box  # [x_min, y_min, width, height]
                    x_min, y_min, width, height = bbox
                    x_max = x_min + width
                    y_max = y_min + height

                    writer.writerow([image_name, label_id, x_min, y_min, x_max, y_max])
                else:
                    print(f"Warning: Label '{label}' not found in the final mappings. Skipping this annotation.")
            else:
                print(f"Warning: Label '{label}' not in updated labels. Skipping this annotation.")

print(f"Finished writing {annotations_csv}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading the COCO-2017 dataset...


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


Sufficient images already downloaded


...

Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'coco-2017'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'coco-2017'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use



Labels before any processing:
Number of unique labels before processing: 80
Original label: aeroplane
Original label: apple
Original label: backpack
Original label: banana
Original label: baseball bat
Original label: baseball glove
Original label: bear
Original label: bed
Original label: bench
Original label: bicycle
Original label: bird
Original label: boat
Original label: book
Original label: bottle
Original label: bowl
Original label: broccoli
Original label: bus
Original label: cake
Original label: car
Original label: carrot
Original label: cat
Original label: cell phone
Original label: chair
Original label: clock
Original label: couch
Original label: cow
Original label: cup
Original label: diningtable
Original label: dog
Original label: donut
Original label: elephant
Original label: fire hydrant
Original label: fork
Original label: frisbee
Original label: giraffe
Original label: hair drier
Original label: handbag
Original label: horse
Original label: hot dog
Original label: keybo

In [75]:
import os
import csv
import tensorflow as tf
import tensorflow_datasets as tfds
import random
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the base path where you want to save CSVs and other files
base_path = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2'

# Set global seed for reproducibility
random.seed(42)

# Load the Pascal VOC 2012 dataset using TensorFlow Datasets
print("Loading the VOC-2007 dataset...")
voc_dataset, info = tfds.load('voc/2007', split='train[:500]', with_info=True)

# Print dataset information
print(info)

# Ensure directories exist
os.makedirs(base_path, exist_ok=True)

# Prepare paths for CSV files
images_csv = os.path.join(base_path, "voc-train-images-boxable.csv")
annotations_csv = os.path.join(base_path, "voc-train-annotations-bbox.csv")
classes_csv = os.path.join(base_path, "voc-class-descriptions-boxable.csv")

# Function to map class labels to a human-readable format
def map_class_label(label):
    return info.features['objects']['label'].int2str(label)

# 1. Create `voc-train-images-boxable.csv` with image names and file paths
print("\nWriting image paths to CSV...")
with open(images_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ImageName", "ImageURL"])

    for i, example in enumerate(voc_dataset):
        image = example['image']
        image_name = f'image_{i}.jpg'
        image_path = os.path.join(base_path, image_name)
        # Save the image to the path
        tf.keras.preprocessing.image.save_img(image_path, image.numpy())
        writer.writerow([image_name, image_path])
print(f"Finished writing {images_csv}")

# 2. Create `voc-class-descriptions-boxable.csv` with class names and corresponding IDs
print("\nWriting class descriptions to CSV...")
with open(classes_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["LabelName", "ClassName"])

    for i in range(info.features['objects']['label'].num_classes):
        class_name = map_class_label(i)
        writer.writerow([i, class_name])
        print(f"Processed {i} -> {class_name}")
print("Finished writing class-descriptions-boxable.csv")

# 3. Create `voc-train-annotations-bbox.csv` with bounding box annotations
print("\nWriting bounding box annotations to CSV...")
with open(annotations_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ImageID", "LabelName", "XMin", "YMin", "XMax", "YMax"])

    for i, example in enumerate(voc_dataset):
        image_name = f'image_{i}.jpg'
        objects = example['objects']

        for bbox, label in zip(objects['bbox'], objects['label']):
            # Bounding boxes are typically stored as [ymin, xmin, ymax, xmax]
            x_min = bbox[1].numpy()  # xmin
            y_min = bbox[0].numpy()  # ymin
            x_max = bbox[3].numpy()  # xmax
            y_max = bbox[2].numpy()  # ymax

            # Instead of saving the class name, save the numeric ID
            label_id = label.numpy()  # Get the numeric ID of the class

            writer.writerow([image_name, label_id, x_min, y_min, x_max, y_max])
print(f"Finished writing {annotations_csv}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading the VOC-2007 dataset...
tfds.core.DatasetInfo(
    name='voc',
    full_name='voc/2007/4.0.0',
    description="""
    This dataset contains the data from the PASCAL Visual Object Classes Challenge,
    corresponding to the Classification and Detection competitions.
    
    In the Classification competition, the goal is to predict the set of labels
    contained in the image, while in the Detection competition the goal is to
    predict the bounding box and label of each individual object.
    annotations.
    """,
    config_description="""
    This dataset contains the data from the PASCAL Visual Object Classes Challenge
    2007, a.k.a. VOC2007.
    
    A total of 9963 images are included in this dataset, where each image
    contains a set of objects, out of 20 different classes, making a total of
    24640 annotated objects.
    
    """,
    h

In [10]:
images_boxable_fname = 'train-images-boxable.csv'
annotations_bbox_fname = 'train-annotations-bbox.csv'
class_descriptions_fname = 'class-descriptions-boxable.csv'
base_path = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2'


# Join the base path with each file name
images_boxable_fname = os.path.join(base_path, images_boxable_fname)
annotations_bbox_fname = os.path.join(base_path, annotations_bbox_fname)
class_descriptions_fname = os.path.join(base_path, class_descriptions_fname)

voc_images_boxable_fname = 'voc-train-images-boxable.csv'
voc_annotations_bbox_fname = 'voc-train-annotations-bbox.csv'
voc_class_descriptions_fname = 'voc-class-descriptions-boxable.csv'


# Join the base path with each file name
voc_images_boxable_fname = os.path.join(base_path, voc_images_boxable_fname)
voc_annotations_bbox_fname = os.path.join(base_path, voc_annotations_bbox_fname)
voc_class_descriptions_fname = os.path.join(base_path, voc_class_descriptions_fname)

In [11]:
images_boxable = pd.read_csv(images_boxable_fname)
images_boxable.head()

,ImageName,ImageURL
0,000000000025.jpg,/root/fiftyone/coco-2017/train/data/0000000000...
1,000000000030.jpg,/root/fiftyone/coco-2017/train/data/0000000000...
2,000000000034.jpg,/root/fiftyone/coco-2017/train/data/0000000000...
3,000000000042.jpg,/root/fiftyone/coco-2017/train/data/0000000000...
4,000000000072.jpg,/root/fiftyone/coco-2017/train/data/0000000000...


In [12]:
voc_images_boxable = pd.read_csv(voc_images_boxable_fname)
voc_images_boxable.head()

,ImageName,ImageURL
0,image_0.jpg,/content/drive/MyDrive/RCNN/Faster_RCNN_for_Op...
1,image_1.jpg,/content/drive/MyDrive/RCNN/Faster_RCNN_for_Op...
2,image_2.jpg,/content/drive/MyDrive/RCNN/Faster_RCNN_for_Op...
3,image_3.jpg,/content/drive/MyDrive/RCNN/Faster_RCNN_for_Op...
4,image_4.jpg,/content/drive/MyDrive/RCNN/Faster_RCNN_for_Op...


In [13]:
annotations_bbox = pd.read_csv(annotations_bbox_fname)
annotations_bbox.head()


,ImageID,LabelName,XMin,YMin,XMax,YMax
0,000000000025.jpg,19,0.602391,0.140915,0.938281,0.838474
1,000000000025.jpg,19,0.082828,0.836831,0.289125,0.966385
2,000000000030.jpg,22,0.320094,0.072477,0.718344,0.829766
3,000000000034.jpg,18,0.001500,0.047200,0.690922,0.939318
4,000000000042.jpg,12,0.334609,0.086381,0.878766,0.596381


In [14]:
voc_annotations_bbox = pd.read_csv(voc_annotations_bbox_fname)
voc_annotations_bbox.head()

,ImageID,LabelName,XMin,YMin,XMax,YMax
0,image_0.jpg,12,0.043702,0.143750,0.776350,0.970833
1,image_0.jpg,14,0.241645,0.145833,0.606684,0.579167
2,image_0.jpg,12,0.524422,0.600000,0.766067,0.854167
3,image_0.jpg,14,0.501285,0.560417,0.686375,0.739583
4,image_1.jpg,6,0.080000,0.074667,0.812000,0.885333


In [15]:
# Load the annotations and class descriptions CSVs
annotations_df = pd.read_csv(annotations_bbox_fname)
class_descriptions_df = pd.read_csv(class_descriptions_fname)
print(annotations_df['LabelName'].unique())
print(class_descriptions_df)


[19 22 18 12  4  5  3  8 16 20 23  7  1 21  9 17 24 13 11 14 10 15  2  6]
    LabelName    ClassName
0           1       person
1           2      bicycle
2           3          car
3           4    motorbike
4           5    aeroplane
5           6          bus
6           7        train
7           8        truck
8           9         boat
9          10         bird
10         11          cat
11         12          dog
12         13        horse
13         14        sheep
14         15          cow
15         16     elephant
16         17         bear
17         18        zebra
18         19      giraffe
19         20        chair
20         21        couch
21         22  pottedplant
22         23  diningtable
23         24    tvmonitor


In [16]:
#class_descriptions = pd.read_csv(class_descriptions_fname, header=None)
class_descriptions = pd.read_csv(class_descriptions_fname, header=0)
class_descriptions.head()


,LabelName,ClassName
0,1,person
1,2,bicycle
2,3,car
3,4,motorbike
4,5,aeroplane


In [17]:

#class_descriptions = pd.read_csv(class_descriptions_fname, header=None)
voc_class_descriptions = pd.read_csv(voc_class_descriptions_fname, header=0)
voc_class_descriptions.head()

,LabelName,ClassName
0,0,aeroplane
1,1,bicycle
2,2,bird
3,3,boat
4,4,bottle


### Plot Bounding box

Finding images with lesser number of objects so as easy to visualize

In [18]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from skimage import io
import random
def plot_bbox(image_name):
    # Get the image URL from the images_boxable CSV using the ImageName
    img_url = images_boxable.loc[images_boxable["ImageName"] == image_name]['ImageURL'].values[0]
    img = io.imread(img_url)  # Load the image from the URL
    height, width, _ = img.shape
    print(f"Image: {img.shape}")

    # Get the bounding boxes for this image from the annotations_bbox CSV
    bboxs = annotations_bbox[annotations_bbox['ImageID'] == image_name]

    for index, row in bboxs.iterrows():
        # Extract the bounding box coordinates
        xmin = int(row['XMin'] * width)
        xmax = int(row['XMax'] * width)
        ymin = int(row['YMin'] * height)
        ymax = int(row['YMax'] * height)

        # Get the class label for this bounding box
        label_id = row['LabelName']  # This is the numeric label

        # Ensure that label_id matches a row in the class_descriptions_df
        class_series = class_descriptions[class_descriptions['LabelName'] == label_id]

        if not class_series.empty:
            class_name = class_series['ClassName'].values[0]
        else:
            print(f"Class name for label {label_id} not found. Skipping...")
            continue

        print(f"Coordinates: {xmin, ymin}, {xmax, ymax}")

        # Draw the bounding box and label on the image
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, class_name, (xmin, ymin - 10), font, 0.9, (0, 255, 0), 2)

    # Display the image with bounding boxes
    plt.figure(figsize=(15, 10))
    plt.title('Image with Bounding Box')
    plt.imshow(img)
    plt.axis("off")
    plt.show()
# Get the least common ImageNames (or adjust this depending on your actual column names)
least_objects_img_names = annotations_bbox["ImageID"].value_counts().tail(50).index.values

# Plot bounding boxes for a random sample of 5 images
for img_name in random.sample(list(least_objects_img_names), 5):
    plot_bbox(img_name)

FileNotFoundError: No such file: '/root/fiftyone/coco-2017/validation/data/000000167540.jpg'

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV files
images_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-train-images-boxable.csv'
annotations_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-train-annotations-bbox.csv'
classes_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-class-descriptions-boxable.csv'

# Read CSV files into DataFrames
images_df = pd.read_csv(images_csv)
annotations_df = pd.read_csv(annotations_csv)
classes_df = pd.read_csv(classes_csv)

# Ensure LabelName is of the same type in both DataFrames
annotations_df['LabelName'] = annotations_df['LabelName'].astype(str)
classes_df['LabelName'] = classes_df['LabelName'].astype(str)

def plot_bbox(image_name):
    # Get the image path from the images CSV
    img_path = images_df.loc[images_df["ImageName"] == image_name, 'ImageURL'].values[0]

    # Load the image
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    print(f"Image: {img.shape}")

    # Get the bounding boxes for this image from the annotations CSV
    bboxs = annotations_df[annotations_df['ImageID'] == image_name]

    for index, row in bboxs.iterrows():
        # Extract the bounding box coordinates
        xmin = int(row['XMin'] * width)
        xmax = int(row['XMax'] * width)
        ymin = int(row['YMin'] * height)
        ymax = int(row['YMax'] * height)

        # Get the class label for this bounding box
        label_name = row['LabelName']
        class_series = classes_df[classes_df['LabelName'] == label_name]

        # Check if the class name exists in the class descriptions CSV
        if not class_series.empty:
            class_name = class_series['ClassName'].values[0]
        else:
            class_name = "Unknown"  # Handle the case where no class name is found
            print(f"Warning: No matching class name found for LabelName: {label_name}")

        print(f"Coordinates: {xmin, ymin}, {xmax, ymax}, Class: {class_name}")

        # Draw the bounding box and label on the image
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, class_name, (xmin, ymin - 10), font, 0.9, (0, 255, 0), 2)

    # Display the image with bounding boxes
    plt.figure(figsize=(15, 10))
    plt.title('Image with Bounding Box')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for plotting
    plt.axis("off")
    plt.show()

# Get some image names to visualize (adjust based on your dataset)
image_names = images_df["ImageName"].sample(5).values

# Plot bounding boxes for a few images
for img_name in image_names:
    plot_bbox(img_name)

### Download images

We need to save the images with filename as `image id` with jpg extension

In [12]:
import os
import requests
import cv2
from skimage import io
import fiftyone as fo
import fiftyone.zoo as foz

# Set paths and configuration
base_path = '/content/drive/My Drive/datasets'
download_base_path = os.path.join(base_path, 'coco')  # Base path to save downloaded images

# Get unique classes in the dataset
classes = coco_dataset.distinct("ground_truth.detections.label")

# Prepare mappings from class to image URLs
class_mappings = {cls: {} for cls in classes}

for sample in coco_dataset:
    image_path = sample.filepath
    image_id = os.path.splitext(os.path.basename(image_path))[0]
    for detection in sample.ground_truth.detections:
        class_name = detection.label
        if class_name in class_mappings:
            class_mappings[class_name][image_id] = image_path

# Download images and save them
for cls_name, mappings in class_mappings.items():
    n_issues = 0
    # Create the directory for the class if it doesn't exist
    class_dir = os.path.join(download_base_path, cls_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    for img_id, local_img_path in mappings.items():
        try:
            # Load the image using OpenCV
            img = cv2.imread(local_img_path)
            if img is None:
                raise Exception(f"Image {local_img_path} could not be read.")

            # Save the image to the appropriate directory
            saved_path = os.path.join(class_dir, img_id + ".jpg")
            cv2.imwrite(saved_path, img)
        except Exception as e:
            print(f"Failed to process {local_img_path}: {e}")
            n_issues += 1

    print(f"Image issues for {cls_name}: {n_issues}")

Image issues for aeroplane: 0
Image issues for bear: 0
Image issues for bicycle: 0
Image issues for bird: 0
Image issues for boat: 0
Image issues for bus: 0
Image issues for car: 0
Image issues for cat: 0
Image issues for chair: 0
Image issues for couch: 0
Image issues for cow: 0
Image issues for diningtable: 0
Image issues for dog: 0
Image issues for elephant: 0
Image issues for giraffe: 0
Image issues for horse: 0
Image issues for motorbike: 0
Image issues for person: 0
Image issues for pottedplant: 0
Image issues for sheep: 0
Image issues for train: 0
Image issues for truck: 0
Image issues for tvmonitor: 0
Image issues for zebra: 0


In [80]:
import os
import cv2
import pandas as pd
from tqdm import tqdm

# File paths for CSVs
images_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-train-images-boxable.csv'
annotations_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-train-annotations-bbox.csv'
classes_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-class-descriptions-boxable.csv'

# Base path where images will be saved, organized by class
save_base_path = '/content/drive/MyDrive/datasets/voc'

# Ensure the base directory exists
os.makedirs(save_base_path, exist_ok=True)

# Load the CSV files into DataFrames
images_df = pd.read_csv(images_csv)
annotations_df = pd.read_csv(annotations_csv)
classes_df = pd.read_csv(classes_csv)

# Create a dictionary to map LabelName (from annotations) to ClassName (from class descriptions)
label_to_class = pd.Series(classes_df.ClassName.values, index=classes_df.LabelName.astype(str)).to_dict()

# Convert LabelName column in annotations_df to string to match the class mapping
annotations_df['LabelName'] = annotations_df['LabelName'].astype(str)

# Group annotations by ImageID to process all bounding boxes for each image together
grouped_annotations = annotations_df.groupby('ImageID')

# Iterate through images in the dataset
for image_name in tqdm(images_df['ImageName'].unique()):
    try:
        # Get the image path from the images CSV
        img_path = images_df.loc[images_df['ImageName'] == image_name, 'ImageURL'].values[0]

        # Load the image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not load image {img_path}")
            continue

        # Get the image's dimensions
        height, width, _ = img.shape

        # Get all annotations for the current image
        if image_name in grouped_annotations.groups:
            bboxs = grouped_annotations.get_group(image_name)
        else:
            print(f"No annotations found for {image_name}")
            continue

        # Loop through the bounding boxes and their labels
        for index, row in bboxs.iterrows():
            # Extract the bounding box coordinates
            xmin = int(row['XMin'] * width)
            ymin = int(row['YMin'] * height)
            xmax = int(row['XMax'] * width)
            ymax = int(row['YMax'] * height)

            # Get the class name for this annotation
            label_name = str(row['LabelName'])  # LabelName from annotations CSV
            class_name = label_to_class.get(label_name, 'Unknown')

            if class_name == 'Unknown':
                print(f"Warning: No class name found for LabelName {label_name}")
                continue

            # Save the image in a class-specific directory
            class_dir = os.path.join(save_base_path, class_name)
            os.makedirs(class_dir, exist_ok=True)
            save_path = os.path.join(class_dir, image_name)

            # Check if image already saved (to avoid saving multiple times for multiple bboxes)
            if not os.path.exists(save_path):
                cv2.imwrite(save_path, img)
    except Exception as e:
        print(f"Error processing {image_name}: {e}")

print("Finished processing and saving images by class.")

100%|██████████| 500/500 [00:12<00:00, 39.52it/s]

Finished processing and saving images by class.


### Dataset format for Faster-RCNN code

(fname_path, xmin, xmax, ymin, ymax, class_name)

train: 0.8
validation: 0.2

In [2]:
# Paths for saving images in Google Drive
train_path = '/content/drive/MyDrive/datasets/coco/train'
val_path = '/content/drive/MyDrive/datasets/coco/val'
test_path= '/content/drive/MyDrive/datasets/voc'

In [17]:
!mkdir train val test

mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘val’: File exists


In [18]:
random.seed(42)

In [7]:
drive.mount('/content/drive')

path = '/content/drive/My Drive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/train'
!mkdir path


NameError: name 'drive' is not defined

In [82]:
import os
import random
import pandas as pd
from PIL import Image
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths for saving images in Google Drive
train_path = '/content/drive/MyDrive/datasets/coco/train'
val_path = '/content/drive/MyDrive/datasets/coco/val'

# Ensure the train and val directories exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# Example: Let's assume you have loaded a FiftyOne dataset
import fiftyone as fo

# Load the FiftyOne dataset (replace with actual dataset loading code)
dataset = coco_dataset  # Adjust with your dataset name

# Get all samples and shuffle them for random split
all_samples = list(dataset)  # Convert to list to shuffle
random.shuffle(all_samples)

# Split dataset into train and validation sets (80% train, 20% val)
split_index = int(len(all_samples) * 0.8)
train_samples = all_samples[:split_index]
val_samples = all_samples[split_index:]

# Function to save images from in-memory variable
def save_image_from_sample(sample, save_path):
    # Get image filepath and save the image
    img_path = sample.filepath  # The image is stored in the file path
    image = Image.open(img_path)

    # Save the image to the specified path
    image.save(save_path)

# Save train images
for sample in train_samples:
    img_name = os.path.basename(sample.filepath)  # Get image name
    save_path = os.path.join(train_path, img_name)

    print(f"Saving {img_name} to {save_path}")
    save_image_from_sample(sample, save_path)

# Save validation images
for sample in val_samples:
    img_name = os.path.basename(sample.filepath)  # Get image name
    save_path = os.path.join(val_path, img_name)

    print(f"Saving {img_name} to {save_path}")
    save_image_from_sample(sample, save_path)

print("Finished saving images to train and val directories.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving 000000013671.jpg to /content/drive/MyDrive/datasets/coco/train/000000013671.jpg
Saving 000000008595.jpg to /content/drive/MyDrive/datasets/coco/train/000000008595.jpg
Saving 000000018775.jpg to /content/drive/MyDrive/datasets/coco/train/000000018775.jpg
Saving 000000208901.jpg to /content/drive/MyDrive/datasets/coco/train/000000208901.jpg
Saving 000000297396.jpg to /content/drive/MyDrive/datasets/coco/train/000000297396.jpg
Saving 000000004624.jpg to /content/drive/MyDrive/datasets/coco/train/000000004624.jpg
Saving 000000015953.jpg to /content/drive/MyDrive/datasets/coco/train/000000015953.jpg
Saving 000000299609.jpg to /content/drive/MyDrive/datasets/coco/train/000000299609.jpg
Saving 000000011525.jpg to /content/drive/MyDrive/datasets/coco/train/000000011525.jpg
Saving 000000011697.jpg to /content/drive/MyDrive/datasets/coco/train/000000011697.jpg
S

In [83]:
!ls /content/drive/MyDrive/datasets/coco/train
!ls /content/drive/MyDrive/datasets/coco/val

000000000025.jpg  000000009709.jpg  000000018647.jpg  000000115118.jpg	000000343934.jpg
000000000030.jpg  000000009729.jpg  000000018654.jpg  000000115885.jpg	000000344059.jpg
000000000034.jpg  000000009830.jpg  000000018656.jpg  000000116362.jpg	000000344100.jpg
000000000042.jpg  000000009843.jpg  000000018658.jpg  000000116589.jpg	000000344816.jpg
000000000072.jpg  000000009865.jpg  000000018670.jpg  000000118209.jpg	000000344909.jpg
000000000073.jpg  000000009866.jpg  000000018702.jpg  000000118594.jpg	000000345261.jpg
000000000081.jpg  000000009898.jpg  000000018736.jpg  000000119038.jpg	000000346638.jpg
000000000094.jpg  000000009914.jpg  000000018737.jpg  000000119828.jpg	000000346905.jpg
000000000138.jpg  000000009919.jpg  000000018750.jpg  000000121586.jpg	000000347254.jpg
000000000142.jpg  000000009946.jpg  000000018766.jpg  000000121591.jpg	000000347456.jpg
000000000144.jpg  000000010008.jpg  000000018771.jpg  000000122927.jpg	000000347664.jpg
000000000154.jpg  000000010037.j

In [37]:
import os
import pandas as pd
from PIL import Image
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path where the VOC dataset with class-specific directories is located
voc_dataset_path = '/content/drive/MyDrive/datasets/voc'

# Path for saving test images
test_path = '/content/drive/MyDrive/datasets/voc/test'

# Ensure the test directory exists
os.makedirs(test_path, exist_ok=True)

# Load the dataset (replace this with your actual test dataset location)
# Assuming test_df contains image file names under 'FileName'
test_df = pd.read_csv('/content/drive/MyDrive/datasets/voc/test.csv')

# Function to find and save images from class-specific directories
def save_image_from_class_dirs(img_name, class_name, save_path):
    # Construct the full path to the image based on its class
    img_path = os.path.join(voc_dataset_path, class_name, img_name)

    # Open and save the image to the test directory
    try:
        image = Image.open(img_path)
        image.save(save_path)
        print(f"Saved {img_name} from class {class_name} to {save_path}")
    except Exception as e:
        print(f"Error saving {img_name}: {e}")

# Save test images from class-specific directories
for idx, row in test_df.iterrows():
    img_name = row['FileName']
    class_name = row['ClassName']  # Assumes class name is in 'ClassName' column
    save_path = os.path.join(test_path, img_name)

    save_image_from_class_dirs(img_name, class_name, save_path)

print("Finished saving images to the test directory.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved image_9.jpg from class aeroplane to /content/drive/MyDrive/datasets/voc/test/image_9.jpg
Saved image_23.jpg from class tvmonitor to /content/drive/MyDrive/datasets/voc/test/image_23.jpg
Saved image_23.jpg from class person to /content/drive/MyDrive/datasets/voc/test/image_23.jpg
Saved image_23.jpg from class person to /content/drive/MyDrive/datasets/voc/test/image_23.jpg
Saved image_23.jpg from class chair to /content/drive/MyDrive/datasets/voc/test/image_23.jpg
Saved image_36.jpg from class dog to /content/drive/MyDrive/datasets/voc/test/image_36.jpg
Saved image_42.jpg from class car to /content/drive/MyDrive/datasets/voc/test/image_42.jpg
Saved image_56.jpg from class cat to /content/drive/MyDrive/datasets/voc/test/image_56.jpg
Saved image_56.jpg from class person to /content/drive/MyDrive/datasets/voc/test/image_56.jpg
Saved image_89.jpg from class b

In [84]:
import pandas as pd

# Load CSV files
#annotations_bbox = pd.read_csv('train-annotations-bbox.csv')
#images_boxable = pd.read_csv('train-images-boxable.csv')

# Check if all ImageIDs in annotations_bbox have a matching ImageName in images_boxable
unmatched_image_ids = annotations_bbox[~annotations_bbox['ImageID'].isin(images_boxable['ImageName'])]

if len(unmatched_image_ids) > 0:
    print(f"Warning: {len(unmatched_image_ids)} ImageIDs do not match any ImageNames:")
    print(unmatched_image_ids['ImageID'].unique())
else:
    print("All ImageIDs have matching ImageNames.")

# Check the first few rows of both DataFrames for verification
print("\nSample of annotations_bbox:")
print(annotations_bbox.head())

print("\nSample of images_boxable:")
print(images_boxable.head())

All ImageIDs have matching ImageNames.

Sample of annotations_bbox:
            ImageID  LabelName      XMin      YMin      XMax      YMax
0  000000000025.jpg         19  0.602391  0.140915  0.938281  0.838474
1  000000000025.jpg         19  0.082828  0.836831  0.289125  0.966385
2  000000000030.jpg         22  0.320094  0.072477  0.718344  0.829766
3  000000000034.jpg         18  0.001500  0.047200  0.690922  0.939318
4  000000000042.jpg         12  0.334609  0.086381  0.878766  0.596381

Sample of images_boxable:
          ImageName                                           ImageURL
0  000000000025.jpg  /root/fiftyone/coco-2017/train/data/0000000000...
1  000000000030.jpg  /root/fiftyone/coco-2017/train/data/0000000000...
2  000000000034.jpg  /root/fiftyone/coco-2017/train/data/0000000000...
3  000000000042.jpg  /root/fiftyone/coco-2017/train/data/0000000000...
4  000000000072.jpg  /root/fiftyone/coco-2017/train/data/0000000000...


The expected number of training images and validation images should be 24 and 6 respectively.

However, there might be some overlap images which appear in two or three classes simultaneously. For instance, an image might be a person walking on the street and there are several cars in the street

In [19]:
import os
import pandas as pd
import sys

# Load CSV files
#annotations_bbox = pd.read_csv('train-annotations-bbox.csv')
#images_boxable = pd.read_csv('train-images-boxable.csv')
#class_descriptions = pd.read_csv('class-descriptions-boxable.csv')

# Ensure `train_df` is initialized as an empty DataFrame with the necessary columns
train_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
train_path = '/content/drive/MyDrive/datasets/coco/train'
# List all images in the train directory
train_imgs = os.listdir(train_path)
train_imgs = [name for name in train_imgs if not name.startswith('.')]

# Add a check to match ImageID with ImageName by appending '.jpg'
annotations_bbox['ImageID'] = annotations_bbox['ImageID'].apply(lambda x: f"{x}.jpg" if not x.endswith('.jpg') else x)

# Parse each image and filter annotations
for i in range(len(train_imgs)):
    sys.stdout.write(f'Parse train_imgs {i}; Number of boxes: {len(train_df)}\r')
    sys.stdout.flush()

    img_name = train_imgs[i]
    img_id = img_name  # Now keeping the extension since ImageID in annotations has '.jpg'

    # Debug: Print the image ID being processed
    print(f"Processing image: {img_name}, Image ID: {img_id}")

    # Get the relevant bounding boxes from annotations_bbox using the ImageID
    tmp_df = annotations_bbox[annotations_bbox['ImageID'] == img_id]

    # Debug: Print number of matching annotations found
    print(f"Found {len(tmp_df)} annotations for Image ID: {img_id}")

    for index, row in tmp_df.iterrows():
        label_name = row['LabelName']

        # Match the label name to the class description to get the class name
        matching_class = class_descriptions[class_descriptions['LabelName'] == label_name]

        # Ensure that a matching class was found before proceeding
        if len(matching_class) == 0:
            print(f"No matching class name found for LabelName: {label_name}")
            continue

        class_name = matching_class['ClassName'].values[0]

        # Create a new DataFrame row for the current bounding box
        new_row = pd.DataFrame({
            'FileName': [img_name],
            'XMin': [row['XMin']],
            'XMax': [row['XMax']],
            'YMin': [row['YMin']],
            'YMax': [row['YMax']],
            'ClassName': [class_name]
        })

        # Concatenate the new row to the train_df
        train_df = pd.concat([train_df, new_row], ignore_index=True)

# Optionally save the train_df to a CSV or continue processing
train_df.to_csv("/content/drive/MyDrive/datasets/coco/val/train.csv", index=False)

print("Processing complete. Final DataFrame:")
print(train_df.head())

Processing image: 000000526706.jpg, Image ID: 000000526706.jpg
Found 4 annotations for Image ID: 000000526706.jpg
Processing image: 000000306582.jpg, Image ID: 000000306582.jpg
Found 7 annotations for Image ID: 000000306582.jpg
Processing image: 000000021483.jpg, Image ID: 000000021483.jpg
Found 1 annotations for Image ID: 000000021483.jpg
Processing image: 000000022168.jpg, Image ID: 000000022168.jpg
Found 4 annotations for Image ID: 000000022168.jpg
Processing image: 000000291664.jpg, Image ID: 000000291664.jpg
Found 1 annotations for Image ID: 000000291664.jpg
Processing image: 000000022589.jpg, Image ID: 000000022589.jpg
Found 2 annotations for Image ID: 000000022589.jpg
Processing image: 000000322352.jpg, Image ID: 000000322352.jpg
Found 9 annotations for Image ID: 000000322352.jpg
Processing image: 000000003480.jpg, Image ID: 000000003480.jpg
Found 5 annotations for Image ID: 000000003480.jpg
Processing image: 000000000283.jpg, Image ID: 000000000283.jpg
Found 3 annotations for I

<ipython-input-19-d76ea753866e>:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, new_row], ignore_index=True)


Processing image: 000000023937.jpg, Image ID: 000000023937.jpg
Found 6 annotations for Image ID: 000000023937.jpg
Processing image: 000000004754.jpg, Image ID: 000000004754.jpg
Found 6 annotations for Image ID: 000000004754.jpg
Processing image: 000000521231.jpg, Image ID: 000000521231.jpg
Found 1 annotations for Image ID: 000000521231.jpg
Processing image: 000000014297.jpg, Image ID: 000000014297.jpg
Found 8 annotations for Image ID: 000000014297.jpg
Processing image: 000000101884.jpg, Image ID: 000000101884.jpg
Found 2 annotations for Image ID: 000000101884.jpg
Processing image: 000000573391.jpg, Image ID: 000000573391.jpg
Found 1 annotations for Image ID: 000000573391.jpg
Processing image: 000000472375.jpg, Image ID: 000000472375.jpg
Found 2 annotations for Image ID: 000000472375.jpg
Processing image: 000000023511.jpg, Image ID: 000000023511.jpg
Found 2 annotations for Image ID: 000000023511.jpg
Processing image: 000000270402.jpg, Image ID: 000000270402.jpg
Found 2 annotations for I

In [20]:
import os
import pandas as pd
import sys

# Ensure `val_df` is initialized as an empty DataFrame with the necessary columns
val_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])

# Path to the validation image folder
val_path = '/content/drive/MyDrive/datasets/coco/val'
val_imgs = os.listdir(val_path)
val_imgs = [name for name in val_imgs if not name.startswith('.')]

# Add a check to match ImageID with ImageName by appending '.jpg'
annotations_bbox['ImageID'] = annotations_bbox['ImageID'].apply(lambda x: f"{x}.jpg" if not x.endswith('.jpg') else x)

# Parse each image and filter annotations for validation set
for i in range(len(val_imgs)):
    sys.stdout.write(f'Parse val_imgs {i}; Number of boxes: {len(val_df)}\r')
    sys.stdout.flush()

    img_name = val_imgs[i]
    img_id = img_name  # Keeping the extension since ImageID in annotations has '.jpg'

    # Debug: Print the image ID being processed
    print(f"Processing image: {img_name}, Image ID: {img_id}")

    # Get the relevant bounding boxes from annotations_bbox_val using the ImageID
    tmp_df = annotations_bbox[annotations_bbox['ImageID'] == img_id]

    # Debug: Print number of matching annotations found
    print(f"Found {len(tmp_df)} annotations for Image ID: {img_id}")

    for index, row in tmp_df.iterrows():
        label_name = row['LabelName']

        # Match the label name to the class description to get the class name
        matching_class = class_descriptions[class_descriptions['LabelName'] == label_name]

        # Ensure that a matching class was found before proceeding
        if len(matching_class) == 0:
            print(f"No matching class name found for LabelName: {label_name}")
            continue

        class_name = matching_class['ClassName'].values[0]

        # Create a new DataFrame row for the current bounding box
        new_row = pd.DataFrame({
            'FileName': [img_name],
            'XMin': [row['XMin']],
            'XMax': [row['XMax']],
            'YMin': [row['YMin']],
            'YMax': [row['YMax']],
            'ClassName': [class_name]
        })

        # Concatenate the new row to the val_df
        val_df = pd.concat([val_df, new_row], ignore_index=True)

# Optionally save the val_df to a CSV or continue processing
val_df.to_csv("/content/drive/MyDrive/datasets/coco/val/val.csv", index=False)

print("Processing complete. Final Validation DataFrame:")
print(val_df.head())

Processing image: 000000051309.jpg, Image ID: 000000051309.jpg
Found 4 annotations for Image ID: 000000051309.jpg
Processing image: 000000046804.jpg, Image ID: 000000046804.jpg
Found 1 annotations for Image ID: 000000046804.jpg
Processing image: 000000002139.jpg, Image ID: 000000002139.jpg
Found 1 annotations for Image ID: 000000002139.jpg
Processing image: 000000038825.jpg, Image ID: 000000038825.jpg
Found 2 annotations for Image ID: 000000038825.jpg
Processing image: 000000033707.jpg, Image ID: 000000033707.jpg
Found 2 annotations for Image ID: 000000033707.jpg
Processing image: 000000012062.jpg, Image ID: 000000012062.jpg
Found 3 annotations for Image ID: 000000012062.jpg
Processing image: 000000001393.jpg, Image ID: 000000001393.jpg
Found 2 annotations for Image ID: 000000001393.jpg
Processing image: 000000050896.jpg, Image ID: 000000050896.jpg
Found 1 annotations for Image ID: 000000050896.jpg
Processing image: 000000001224.jpg, Image ID: 000000001224.jpg
Found 2 annotations for I

<ipython-input-20-cc9d2bf09079>:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  val_df = pd.concat([val_df, new_row], ignore_index=True)


Processing image: 000000000597.jpg, Image ID: 000000000597.jpg
Found 7 annotations for Image ID: 000000000597.jpg
Processing image: 000000026564.jpg, Image ID: 000000026564.jpg
Found 1 annotations for Image ID: 000000026564.jpg
Processing image: 000000001510.jpg, Image ID: 000000001510.jpg
Found 0 annotations for Image ID: 000000001510.jpg
Processing image: 000000001408.jpg, Image ID: 000000001408.jpg
Found 14 annotations for Image ID: 000000001408.jpg
Processing image: 000000001622.jpg, Image ID: 000000001622.jpg
Found 1 annotations for Image ID: 000000001622.jpg
Processing image: 000000045229.jpg, Image ID: 000000045229.jpg
Found 9 annotations for Image ID: 000000045229.jpg
Processing image: annotation.txt, Image ID: annotation.txt
Found 0 annotations for Image ID: annotation.txt
Processing image: 000000093965.jpg, Image ID: 000000093965.jpg
Found 2 annotations for Image ID: 000000093965.jpg
Processing image: 000000157390.jpg, Image ID: 000000157390.jpg
Found 0 annotations for Image 

In [38]:
import os
import pandas as pd
import sys

# Ensure `test_df` is initialized as an empty DataFrame with the necessary columns
test_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])

# Path to the VOC test image folder, organized by class directories
test_path = '/content/drive/MyDrive/datasets/voc'

# List valid image extensions (e.g., .jpg, .png)
valid_image_extensions = ['.jpg', '.jpeg', '.png']

# Traverse through each class folder and gather images
for class_folder in os.listdir(test_path):
    class_folder_path = os.path.join(test_path, class_folder)

    if os.path.isdir(class_folder_path):  # Check if it's a directory (i.e., a class folder)
        # List images with valid extensions within each class folder
        test_imgs = [name for name in os.listdir(class_folder_path)
                     if os.path.splitext(name)[1].lower() in valid_image_extensions]

        # Parse each image and filter annotations for the test set
        for i, img_name in enumerate(test_imgs):
            sys.stdout.write(f'Parsing test image {i}; Number of boxes: {len(test_df)}\r')
            sys.stdout.flush()

            img_id = img_name  # Keep the extension since ImageID in annotations has '.jpg'

            # Debug: Print the image ID and class being processed
            print(f"Processing image: {img_name}, Image ID: {img_id}, Class: {class_folder}")

            # Get the relevant bounding boxes from annotations_bbox using the ImageID
            tmp_df = voc_annotations_bbox[voc_annotations_bbox['ImageID'] == img_id]

            # Debug: Print number of matching annotations found
            print(f"Found {len(tmp_df)} annotations for Image ID: {img_id}")

            for index, row in tmp_df.iterrows():
                label_name = row['LabelName']

                # Match the label name to the class description to get the class name
                matching_class = voc_class_descriptions[voc_class_descriptions['LabelName'] == label_name]

                # Ensure that a matching class was found before proceeding
                if len(matching_class) == 0:
                    print(f"No matching class name found for LabelName: {label_name}")
                    continue

                class_name = matching_class['ClassName'].values[0]

                # Create a new DataFrame row for the current bounding box
                new_row = pd.DataFrame({
                    'FileName': [img_name],
                    'XMin': [row['XMin']],
                    'XMax': [row['XMax']],
                    'YMin': [row['YMin']],
                    'YMax': [row['YMax']],
                    'ClassName': [class_name]
                })

                # Concatenate the new row to the test_df
                test_df = pd.concat([test_df, new_row], ignore_index=True)

# Optionally save the test_df to a CSV or continue processing
test_df.to_csv("/content/drive/MyDrive/datasets/voc/test.csv", index=False)

print("Processing complete. Final Test DataFrame:")
print(test_df.head())

Processing image: image_9.jpg, Image ID: image_9.jpg, Class: aeroplane
Found 1 annotations for Image ID: image_9.jpg
Processing image: image_23.jpg, Image ID: image_23.jpg, Class: aeroplane
Found 4 annotations for Image ID: image_23.jpg
Processing image: image_36.jpg, Image ID: image_36.jpg, Class: aeroplane
Found 1 annotations for Image ID: image_36.jpg
Processing image: image_42.jpg, Image ID: image_42.jpg, Class: aeroplane
Found 1 annotations for Image ID: image_42.jpg
Processing image: image_56.jpg, Image ID: image_56.jpg, Class: aeroplane
Found 2 annotations for Image ID: image_56.jpg
Processing image: image_89.jpg, Image ID: image_89.jpg, Class: aeroplane
Found 1 annotations for Image ID: image_89.jpg
Processing image: image_95.jpg, Image ID: image_95.jpg, Class: aeroplane
Found 2 annotations for Image ID: image_95.jpg
Processing image: image_150.jpg, Image ID: image_150.jpg, Class: aeroplane
Found 1 annotations for Image ID: image_150.jpg
Processing image: image_177.jpg, Image I

<ipython-input-38-2c968e066c69>:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_df = pd.concat([test_df, new_row], ignore_index=True)


Streaming output truncated to the last 5000 lines.
Found 0 annotations for Image ID: image_2380.jpg
Processing image: image_2415.jpg, Image ID: image_2415.jpg, Class: motorbike
Found 0 annotations for Image ID: image_2415.jpg
Processing image: image_2417.jpg, Image ID: image_2417.jpg, Class: motorbike
Found 0 annotations for Image ID: image_2417.jpg
Processing image: image_2429.jpg, Image ID: image_2429.jpg, Class: motorbike
Found 0 annotations for Image ID: image_2429.jpg
Processing image: image_2453.jpg, Image ID: image_2453.jpg, Class: motorbike
Found 0 annotations for Image ID: image_2453.jpg
Processing image: image_2472.jpg, Image ID: image_2472.jpg, Class: motorbike
Found 0 annotations for Image ID: image_2472.jpg
Processing image: image_63.jpg, Image ID: image_63.jpg, Class: motorbike
Found 2 annotations for Image ID: image_63.jpg
Processing image: image_81.jpg, Image ID: image_81.jpg, Class: motorbike
Found 1 annotations for Image ID: image_81.jpg
Processing image: image_100.jp

In [39]:
train_df.head()
val_df.head()
test_df.head()

,FileName,XMin,XMax,YMin,YMax,ClassName
0,image_9.jpg,0.028,0.974,0.207207,0.657658,aeroplane
1,image_23.jpg,0.002,0.116,0.274752,0.418317,tvmonitor
2,image_23.jpg,0.742,0.952,0.396040,1.000000,person
3,image_23.jpg,0.454,0.730,0.245050,1.000000,person
4,image_23.jpg,0.292,0.526,0.512376,0.905941,chair


In [23]:
import pandas as pd

# Assuming train_df and val_df are already created

# Check for overlapping FileNames (images) between train and validation sets
overlapping_images = pd.merge(train_df, val_df, on='FileName', how='inner', suffixes=('_train', '_val'))

# Print comparison results
if not overlapping_images.empty:
    print(f"Found {len(overlapping_images)} overlapping images between train and validation:")
    print(overlapping_images[['FileName', 'XMin_train', 'YMin_train', 'XMax_train', 'YMax_train',
                              'XMin_val', 'YMin_val', 'XMax_val', 'YMax_val']])
else:
    print("No overlapping images found between train and validation.")

Found 2 overlapping images between train and validation:
           FileName  XMin_train  YMin_train  XMax_train  YMax_train  XMin_val  \
0  000000039670.jpg    0.076234    0.158297    0.894625    0.707625  0.076234   
1  000000050165.jpg    0.181594    0.573039    0.708250    0.761787  0.181594   

   YMin_val  XMax_val  YMax_val  
0  0.158297  0.894625  0.707625  
1  0.573039  0.708250  0.761787  


In [24]:
train_df.shape

(360, 6)

Let's test if they work fine by plotting the bounding box for the above 5 images

In [40]:
train_img_ids = train_df["FileName"].head().str.split(".").str[0].unique()
val_img_ids = val_df["FileName"].head().str.split(".").str[0].unique()


In [ ]:
# Extract the image IDs from the 'FileName' column by removing the extensions
test_img_ids = test_df["FileName"].apply(lambda x: os.path.splitext(x)[0]).unique()
# Check the extracted test_img_ids
print(test_img_ids)

import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd

# Load your CSV files
images_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-train-images-boxable.csv'
annotations_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-train-annotations-bbox.csv'
classes_csv = '/content/drive/MyDrive/RCNN/Faster_RCNN_for_Open_Images_Dataset_Keras-master 2/voc-class-descriptions-boxable.csv'

# Read CSV files into DataFrames
images_df = pd.read_csv(images_csv)
annotations_df = pd.read_csv(annotations_csv)
classes_df = pd.read_csv(classes_csv)

# Ensure LabelName is of the same type in both DataFrames
annotations_df['LabelName'] = annotations_df['LabelName'].astype(str)
classes_df['LabelName'] = classes_df['LabelName'].astype(str)

def plot_bbox(image_name):
    # Get the image path from the images CSV
    img_path = images_df.loc[images_df["ImageName"] == image_name, 'ImageURL'].values
    if len(img_path) == 0:
        print(f"Image {image_name} not found in images_df")
        return
    img_path = img_path[0]

    # Load the image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not load image at path {img_path}")
        return

    height, width, _ = img.shape
    print(f"Image: {img.shape}")

    # Get the bounding boxes for this image from the annotations CSV
    bboxs = annotations_df[annotations_df['ImageID'] == image_name]

    for index, row in bboxs.iterrows():
        # Extract the bounding box coordinates
        xmin = int(row['XMin'] * width)
        xmax = int(row['XMax'] * width)
        ymin = int(row['YMin'] * height)
        ymax = int(row['YMax'] * height)

        # Get the class label for this bounding box
        label_name = row['LabelName']
        class_series = classes_df[classes_df['LabelName'] == label_name]

        # Check if the class name exists in the class descriptions CSV
        if not class_series.empty:
            class_name = class_series['ClassName'].values[0]
        else:
            class_name = "Unknown"  # Handle the case where no class name is found
            print(f"Warning: No matching class name found for LabelName: {label_name}")

        print(f"Coordinates: {xmin, ymin}, {xmax, ymax}, Class: {class_name}")

        # Draw the bounding box and label on the image
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, class_name, (xmin, ymin - 10), font, 0.9, (0, 255, 0), 2)

    # Display the image with bounding boxes
    plt.figure(figsize=(15, 10))
    plt.title('Image with Bounding Box')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for plotting
    plt.axis("off")
    plt.show()

# Get test image names to visualize
for img_id in test_img_ids:
    plot_bbox(f"{img_id}.jpg")

In [47]:
import imageio as io # Make sure to import the imageio module


for img_id in train_img_ids:
  plot_bbox(f"{img_id}.jpg")


for img_id in val_img_ids:
  plot_bbox(f"{img_id}.jpg")

Image 000000526706.jpg not found in images_df
Image 000000306582.jpg not found in images_df
Image 000000051309.jpg not found in images_df
Image 000000046804.jpg not found in images_df


**This looks fine to me!!**

In [48]:
train_df.to_csv('/content/drive/MyDrive/datasets/coco/train/train.csv')
val_df.to_csv('/content/drive/MyDrive/datasets/coco/val/val.csv')
test_df.to_csv('/content/drive/MyDrive/datasets/voc/val.csv')

### Write train.csv to annotation.txt

In [49]:
import os
import pandas as pd
import cv2

# Load the training DataFrame
train_df = pd.read_csv('/content/drive/MyDrive/datasets/coco/train/train.csv')

# Set the image base path to match your previous code
image_base_path = '/content/drive/MyDrive/datasets/coco/train'

# Define the path where the annotation.txt file will be saved
annotation_file = '/content/drive/MyDrive/datasets/coco/train/annotation.txt'

# Create and write to the annotation.txt file
with open(annotation_file, "w+") as f:
    for idx, row in train_df.iterrows():
        img_path = os.path.join(image_base_path, row['FileName'])
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warning: Image {row['FileName']} not found at {img_path}")
            continue

        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)

        # Full image path for saving in the annotation.txt file
        class_name = row['ClassName']
        f.write(f"{img_path},{x1},{y1},{x2},{y2},{class_name}\n")

print("Annotation file created successfully.")

# Check if the file was created
if os.path.exists(annotation_file):
    print(f"File '{annotation_file}' created successfully.")

    # Optionally, read back the file to confirm its content
    with open(annotation_file, "r") as f:
        print("Here are the first few lines of the file:")
        for i, line in enumerate(f):
            print(line.strip())  # Print the line without extra newlines
            if i >= 4:  # Stop after printing the first 5 lines
                break
else:
    print(f"Error: File '{annotation_file}' was not created.")

Annotation file created successfully.
File '/content/drive/MyDrive/datasets/coco/train/annotation.txt' created successfully.
Here are the first few lines of the file:
/content/drive/MyDrive/datasets/coco/train/000000526706.jpg,610,187,640,262,cow
/content/drive/MyDrive/datasets/coco/train/000000526706.jpg,171,238,316,346,cow
/content/drive/MyDrive/datasets/coco/train/000000526706.jpg,354,183,544,315,cow
/content/drive/MyDrive/datasets/coco/train/000000526706.jpg,109,190,364,346,cow
/content/drive/MyDrive/datasets/coco/train/000000306582.jpg,434,247,553,335,cow


In [50]:
import os
import pandas as pd
import cv2

# Load the validation DataFrame
val_df = pd.read_csv('/content/drive/MyDrive/datasets/coco/val/val.csv')

# Set the image base path to match your previous code
image_base_path = '/content/drive/MyDrive/datasets/coco/val'

# Define the path where the annotation.txt file will be saved
annotation_file = '/content/drive/MyDrive/datasets/coco/val/annotation.txt'

# Create and write to the annotation.txt file
with open(annotation_file, "w+") as f:
    for idx, row in val_df.iterrows():
        img_path = os.path.join(image_base_path, row['FileName'])
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warning: Image {row['FileName']} not found at {img_path}")
            continue

        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)

        # Full image path for saving in the annotation.txt file
        class_name = row['ClassName']
        f.write(f"{img_path},{x1},{y1},{x2},{y2},{class_name}\n")

print("Annotation file created successfully.")

# Check if the file was created
if os.path.exists(annotation_file):
    print(f"File '{annotation_file}' created successfully.")

    # Optionally, read back the file to confirm its content
    with open(annotation_file, "r") as f:
        print("Here are the first few lines of the file:")
        for i, line in enumerate(f):
            print(line.strip())  # Print the line without extra newlines
            if i >= 4:  # Stop after printing the first 5 lines
                break
else:
    print(f"Error: File '{annotation_file}' was not created.")

Annotation file created successfully.
File '/content/drive/MyDrive/datasets/coco/val/annotation.txt' created successfully.
Here are the first few lines of the file:
/content/drive/MyDrive/datasets/coco/val/000000051309.jpg,10,162,209,273,horse
/content/drive/MyDrive/datasets/coco/val/000000051309.jpg,67,46,343,271,horse
/content/drive/MyDrive/datasets/coco/val/000000051309.jpg,205,105,351,270,horse
/content/drive/MyDrive/datasets/coco/val/000000051309.jpg,310,81,622,272,horse
/content/drive/MyDrive/datasets/coco/val/000000046804.jpg,223,62,459,364,sheep


In [51]:
import os
import pandas as pd
import cv2

# Load the test DataFrame
test_df = pd.read_csv('/content/drive/MyDrive/datasets/voc/test.csv')

# Set the image base path for the test images (assumed to be organized by class)
image_base_path = '/content/drive/MyDrive/datasets/voc'

# Define the path where the annotation.txt file will be saved for test data
annotation_file = '/content/drive/MyDrive/datasets/voc/annotation.txt'

# Create and write to the annotation.txt file
with open(annotation_file, "w+") as f:
    for idx, row in test_df.iterrows():
        class_folder = row['ClassName']  # Assuming ClassName is used as folder name
        img_path = os.path.join(image_base_path, class_folder, row['FileName'])

        img = cv2.imread(img_path)

        if img is None:
            print(f"Warning: Image {row['FileName']} not found at {img_path}")
            continue

        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)

        # Full image path for saving in the annotation.txt file
        class_name = row['ClassName']
        f.write(f"{img_path},{x1},{y1},{x2},{y2},{class_name}\n")

print("Annotation file created successfully.")

# Check if the file was created
if os.path.exists(annotation_file):
    print(f"File '{annotation_file}' created successfully.")

    # Optionally, read back the file to confirm its content
    with open(annotation_file, "r") as f:
        print("Here are the first few lines of the file:")
        for i, line in enumerate(f):
            print(line.strip())  # Print the line without extra newlines
            if i >= 4:  # Stop after printing the first 5 lines
                break
else:
    print(f"Error: File '{annotation_file}' was not created.")

Annotation file created successfully.
File '/content/drive/MyDrive/datasets/voc/annotation.txt' created successfully.
Here are the first few lines of the file:
/content/drive/MyDrive/datasets/voc/aeroplane/image_9.jpg,14,68,487,219,aeroplane
/content/drive/MyDrive/datasets/voc/tvmonitor/image_23.jpg,1,110,58,169,tvmonitor
/content/drive/MyDrive/datasets/voc/person/image_23.jpg,371,159,476,404,person
/content/drive/MyDrive/datasets/voc/person/image_23.jpg,227,98,365,404,person
/content/drive/MyDrive/datasets/voc/chair/image_23.jpg,146,207,263,366,chair
